## Задание  
> Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:
Наименование вакансии.
Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
Ссылку на саму вакансию.
Сайт, откуда собрана вакансия.
По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.

### Для парсинга выбран сайт  HH.

In [ ]:
from pprint import pprint
import requests
from time import sleep
from bs4 import BeautifulSoup as bs

In [ ]:
URL_hh = 'https://hh.ru/search/vacancy'
#URL_hh = 'https://hh.ru/search/vacancy?text=python&sala&area=1&ored_clusters=true&enable_snippets=true'
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
headers = {'User-Agent': USER_AGENT}
params = {'enable_snippets':'true','text':'python', 'area':'1','ored_clusters':'true', 'enable_snippets':'true'}
response = requests.get(URL_hh, headers = headers,params=params)
dom = bs(response.text,'html.parser') # 
job_list = dom.find_all('div', {'class':'vacancy-serp-item__layout',}) #Ищем все уникальные теги, содержащие инф. о вакансиях
params['page']='0'
params

{'enable_snippets': 'true',
 'text': 'python',
 'area': '1',
 'ored_clusters': 'true',
 'page': '0'}

In [ ]:
len(job_list)

20

In [ ]:
def sort_salary(vac_salary_text,min_salary=None, max_salary=None, currency_salary=None):
  list_salary = vac_salary_text.replace('\u202f', '').split()
  for i in range(len(list_salary) - 1):
    if list_salary[i] == 'от':
      min_salary = int(list_salary[i + 1])
    elif list_salary[i] == 'до':
      max_salary = int(list_salary[i + 1])
    elif list_salary[i] == '–':
      min_salary = int(list_salary[i - 1])
      max_salary = int(list_salary[i + 1])
  currency_salary = list_salary[-1]

  return min_salary, max_salary, currency_salary

In [ ]:
def parse_hadH(url,headers, params, result = [], page = 0):
    params['page'] = page
    headers = headers
    response = requests.get(URL_hh, headers = headers,params=params)
    if response.status_code != 200:
        print('поиск закончен')
        return False
    dom = bs(response.text,'html.parser') 
    job_list = dom.find_all('div', {'class':'vacancy-serp-item__layout',}) #Ищем все уникальные теги, содержащие инф. о вакансиях
    #На странице помещается 20 вакансий. Если длина списка равна 20, то все найденные теги уникальны   
    if(len(job_list) <=20):
              print('все найденные теги уникальны')
    for vac in job_list:
        sleep(1)
        vac_name = vac.find('a').text
        vac_salary = vac.find('span', {'class', 'bloko-header-section-3'})
        if vac_salary:
            vac_salary = vac_salary.text
            min_salary, max_salary, currency_salary = sort_salary(vac_salary)
        else:
            min_salary, max_salary, currency_salary = None, None, None
        vac_link = vac.find('a')['href']
        result.append(   {'vac_name': vac_name,'vac_salary': vac_salary, 'min_salary': min_salary,'max_salary': max_salary,
                          'currency_salary': currency_salary,'vac_link': vac_link,'vacancy_source': 'hh.ru'})
    return result

In [ ]:
pprint(parse_hadH(URL_hh,headers, params,page = 1)) # Вакансии на первой странице
pprint(parse_hadH(URL_hh,headers, params,page = 2)) # Вакансии на второй страницe

все найденные теги уникальны
[{'currency_salary': None,
  'max_salary': None,
  'min_salary': None,
  'vac_link': 'https://hh.ru/vacancy/76088946?from=vacancy_search_list&query=python',
  'vac_name': 'Программист Python',
  'vac_salary': None,
  'vacancy_source': 'hh.ru'},
 {'currency_salary': None,
  'max_salary': None,
  'min_salary': None,
  'vac_link': 'https://hh.ru/vacancy/77088738?from=vacancy_search_list&query=python',
  'vac_name': 'Python Backend Developer (Django)',
  'vac_salary': None,
  'vacancy_source': 'hh.ru'},
 {'currency_salary': None,
  'max_salary': None,
  'min_salary': None,
  'vac_link': 'https://hh.ru/vacancy/76290598?from=vacancy_search_list&query=python',
  'vac_name': 'Менеджер по обработке данных (SPSS, SAS, Python)',
  'vac_salary': None,
  'vacancy_source': 'hh.ru'},
 {'currency_salary': 'USD',
  'max_salary': 4000,
  'min_salary': 3000,
  'vac_link': 'https://hh.ru/vacancy/74736547?from=vacancy_search_list&query=python',
  'vac_name': 'Data Analyst (Remo

все найденные теги уникальны
[{'currency_salary': None,
  'max_salary': None,
  'min_salary': None,
  'vac_link': 'https://hh.ru/vacancy/76088946?from=vacancy_search_list&query=python',
  'vac_name': 'Программист Python',
  'vac_salary': None,
  'vacancy_source': 'hh.ru'},
 {'currency_salary': None,
  'max_salary': None,
  'min_salary': None,
  'vac_link': 'https://hh.ru/vacancy/77088738?from=vacancy_search_list&query=python',
  'vac_name': 'Python Backend Developer (Django)',
  'vac_salary': None,
  'vacancy_source': 'hh.ru'},
 {'currency_salary': None,
  'max_salary': None,
  'min_salary': None,
  'vac_link': 'https://hh.ru/vacancy/76290598?from=vacancy_search_list&query=python',
  'vac_name': 'Менеджер по обработке данных (SPSS, SAS, Python)',
  'vac_salary': None,
  'vacancy_source': 'hh.ru'},
 {'currency_salary': 'USD',
  'max_salary': 4000,
  'min_salary': 3000,
  'vac_link': 'https://hh.ru/vacancy/74736547?from=vacancy_search_list&query=python',
  'vac_name': 'Data Analyst (Remo